In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%%writefile run_inference_timed.py

import pandas as pd
import numpy as np
import os
import joblib
import time
import warnings
from scipy.signal import butter, filtfilt, welch
from tqdm import tqdm

# --- Configuration ---
RAW_DATA_PATH = '/content/drive/MyDrive/aic3/unzipped'
MODEL_CHECKPOINT_PATH = '/content/drive/MyDrive/aic3/ssvep_model_artifacts'
SUBMISSION_FILE_PATH = '/content/drive/MyDrive/aic3/ssvep_submission.csv'
TEST_CSV_PATH = os.path.join(RAW_DATA_PATH, 'test.csv')
TASK = 'SSVEP'

# --- Signal Processing Constants ---
EEG_CHANNELS = ['PO7', 'PZ', 'OZ', 'PO8']
SAMPLING_RATE = 250
TRIAL_DURATION = 7.0
TRIM_START_S = 1.0
TARGET_FREQS = [7, 8, 10, 13]

warnings.filterwarnings('ignore')

# --- Core Preprocessing Functions (Unchanged) ---

def load_test_trial_data(row, base_path):
    """Loads a single trial's EEG data for the test set."""
    subject_folder = row['subject_id']
    session_folder = str(row['trial_session'])
    path = os.path.join(base_path, 'SSVEP', 'test', subject_folder, session_folder, 'EEGdata.csv')
    if not os.path.exists(path):
        print(f"Warning: Test data not found at: {path}. Skipping trial id {row.get('id', 'N/A')}.")
        return None
    eeg_data = pd.read_csv(path).ffill().bfill()
    samples_per_trial = int(TRIAL_DURATION * SAMPLING_RATE)
    trim_samples = int(TRIM_START_S * SAMPLING_RATE)
    start_idx = (int(row['trial']) - 1) * samples_per_trial
    trimmed_start_idx = start_idx + trim_samples
    end_idx = start_idx + samples_per_trial
    return eeg_data.iloc[trimmed_start_idx:end_idx]

def apply_filter(data):
    """Applies a band-pass filter to the EEG data."""
    low, high, order = 4.0, 42.0, 4
    nyq = 0.5 * SAMPLING_RATE
    b, a = butter(order, [low / nyq, high / nyq], btype='band')
    return np.apply_along_axis(lambda x: filtfilt(b, a, x) if not np.all(x == x[0]) else x, 0, data)

def get_psd_features(data):
    """Calculates PSD and SNR features from the EEG data."""
    all_features = []
    nperseg = SAMPLING_RATE * 4
    for i in range(data.shape[1]):
        freqs, psd = welch(data[:, i], fs=SAMPLING_RATE, nperseg=nperseg, nfft=nperseg * 2)
        for f in TARGET_FREQS:
            for h in range(1, 4):
                harmonic_freq = h * f
                target_idx = np.argmin(np.abs(freqs - harmonic_freq))
                noise_indices = np.where((freqs >= harmonic_freq - 3) & (freqs <= harmonic_freq + 3) & (np.abs(freqs - harmonic_freq) > 0.25))[0]
                signal_power = psd[target_idx]
                noise_power = np.mean(psd[noise_indices]) if len(noise_indices) > 0 else 1e-12
                all_features.extend([signal_power, signal_power / noise_power])
    return np.array(all_features)


def main():
    """
    Main inference pipeline with timing focused ONLY on the prediction loop.
    """
    print("--- [1/4] Starting INFERENCE-ONLY Pipeline ---")

    # === UNTIMED SETUP: Load model artifacts ===
    print(f"--- [2/4] Loading model artifacts from: {MODEL_CHECKPOINT_PATH} ---")
    try:
        model = joblib.load(os.path.join(MODEL_CHECKPOINT_PATH, 'ssvep_model.joblib'))
        scaler = joblib.load(os.path.join(MODEL_CHECKPOINT_PATH, 'ssvep_scaler.joblib'))
        selector = joblib.load(os.path.join(MODEL_CHECKPOINT_PATH, 'ssvep_selector.joblib'))
        le = joblib.load(os.path.join(MODEL_CHECKPOINT_PATH, 'ssvep_label_encoder.joblib'))
        print("Model, scaler, selector, and label encoder loaded successfully.")
    except FileNotFoundError as e:
        print(f"FATAL ERROR: A model file was not found. {e}")
        return

    # === UNTIMED SETUP: Load test metadata ===
    print(f"--- [3/4] Loading test data from: {TEST_CSV_PATH} ---")
    test_df = pd.read_csv(TEST_CSV_PATH)
    ssvep_test_df = test_df[test_df['task'] == TASK].reset_index(drop=True)
    print(f"Found {len(ssvep_test_df)} test samples for the '{TASK}' task.")

    # === TIMED PREDICTION PROCESS START ===
    print("--- [4/4] Generating predictions for the test set ---")

    # Start the timer RIGHT BEFORE the loop
    prediction_start_time = time.time()

    results = []
    for _, row in tqdm(ssvep_test_df.iterrows(), total=len(ssvep_test_df), desc="Predicting"):
        trial_data = load_test_trial_data(row, RAW_DATA_PATH)
        if trial_data is None:
            results.append({'id': row['id'], 'label': 'Error_Data_Not_Found'})
            continue

        eeg_signals = trial_data[EEG_CHANNELS].values
        filtered_signals = apply_filter(eeg_signals)
        features = get_psd_features(filtered_signals).reshape(1, -1)

        scaled_features = scaler.transform(features)
        selected_features = selector.transform(scaled_features)
        prediction_encoded = model.predict(selected_features)
        prediction_label = le.inverse_transform(prediction_encoded)[0]

        results.append({'id': row['id'], 'label': prediction_label})

    # Stop the timer RIGHT AFTER the loop
    prediction_end_time = time.time()
    # === TIMED PREDICTION PROCESS END ===

    # --- Final Step: Save Submission and Report Timings ---
    submission_df = pd.DataFrame(results)
    submission_df.to_csv(SUBMISSION_FILE_PATH, index=False)

    total_prediction_time = prediction_end_time - prediction_start_time

    print("\n" + "="*50)
    print("      INFERENCE COMPLETE")
    print("="*50)
    # The printed message now accurately reflects what was measured
    print(f"Total time for prediction loop ONLY: {total_prediction_time:.2f} seconds")
    print(f"Average time per sample: {total_prediction_time / len(ssvep_test_df):.4f} seconds")
    print(f"\nSubmission file saved to: {SUBMISSION_FILE_PATH}")
    print("\nSubmission Head:")
    print(submission_df.head())
    print("="*50)

if __name__ == '__main__':
    main()

Writing run_inference_timed.py


In [5]:
!python run_inference_timed.py

--- [1/4] Starting INFERENCE-ONLY Pipeline ---
--- [2/4] Loading model artifacts from: /content/drive/MyDrive/aic3/ssvep_model_artifacts ---
Model, scaler, selector, and label encoder loaded successfully.
--- [3/4] Loading test data from: /content/drive/MyDrive/aic3/unzipped/test.csv ---
Found 50 test samples for the 'SSVEP' task.
--- [4/4] Generating predictions for the test set ---
Predicting: 100% 50/50 [00:06<00:00,  7.30it/s]

      INFERENCE COMPLETE
Total time for prediction loop ONLY: 6.85 seconds
Average time per sample: 0.1370 seconds

Submission file saved to: /content/drive/MyDrive/aic3/ssvep_submission.csv

Submission Head:
     id     label
0  4951  Backward
1  4952  Backward
2  4953   Forward
3  4954  Backward
4  4955  Backward
